# Feature Engineering: Финальная обработка признаков

Этот ноутбук содержит:
- **Задача 29**: Обработка категориальных признаков
- **Задача 30**: Проверка выбросов
- **Задача 31**: Нормирование/масштабирование (уже сделано в 01_full_data_pipeline.ipynb)
- **Задача 32**: Сохранение features.csv (уже сделано в 01_full_data_pipeline.ipynb)
- **Задача 34**: Preprocessing pipeline
- **Задача 36**: Финальный набор признаков


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import os
import warnings
warnings.filterwarnings('ignore')

# Настройка визуализации
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)

print("=" * 60)
print("FEATURE ENGINEERING: ФИНАЛЬНАЯ ОБРАБОТКА ПРИЗНАКОВ")
print("=" * 60)


## Загрузка данных с признаками


In [ ]:
# Загрузка финального датасета с признаками
FEATURES_FILE = '../data/processed/transactions_with_features.csv'

try:
    df = pd.read_csv(FEATURES_FILE)
    print(f"✅ Данные загружены: {df.shape}")
    print(f"Колонок: {len(df.columns)}")
    print(f"Строк: {len(df)}")
except FileNotFoundError:
    print(f"❌ Файл {FEATURES_FILE} не найден!")
    print("   Сначала запусти 01_full_data_pipeline.ipynb для создания признаков")
    raise


In [ ]:
print("\n--- ЗАДАЧА 29: Обработка категориальных признаков ---\n")

# Поиск всех категориальных признаков (не числовых и не бинарных)
categorical_cols = []
for col in df.columns:
    if col in ['target', 'user_id']:
        continue
    if df[col].dtype == 'object':
        categorical_cols.append(col)
    elif df[col].dtype in [np.int64, np.int32] and df[col].nunique() < 20:
        # Целочисленные колонки с небольшим количеством уникальных значений тоже могут быть категориальными
        if col not in ['hour', 'day_of_week', 'day_of_month', 'month']:
            categorical_cols.append(col)

print(f"Найдено категориальных признаков: {len(categorical_cols)}")
if len(categorical_cols) > 0:
    print("Категориальные колонки:")
    for i, col in enumerate(categorical_cols[:10], 1):
        unique_count = df[col].nunique()
        print(f"  {i}. {col} ({unique_count} уникальных значений)")
    if len(categorical_cols) > 10:
        print(f"  ... и еще {len(categorical_cols) - 10} колонок")

# Обработка категориальных признаков
processed_categorical = []

for col in categorical_cols:
    if col not in df.columns:
        continue
    
    # Заполняем пропуски
    df[col].fillna('UNKNOWN', inplace=True)
    
    unique_count = df[col].nunique()
    
    # Frequency Encoding (для всех категориальных)
    freq_counts = df[col].value_counts()
    df[f'{col}_freq'] = df[col].map(freq_counts) / len(df)
    processed_categorical.append(f'{col}_freq')
    
    # Target Encoding (если есть target) - среднее значение target для каждой категории
    if 'target' in df.columns:
        target_mean = df.groupby(col)['target'].mean()
        df[f'{col}_target_enc'] = df[col].map(target_mean)
        df[f'{col}_target_enc'].fillna(df['target'].mean(), inplace=True)  # Для новых категорий
        processed_categorical.append(f'{col}_target_enc')
    
    # Удаляем исходную категориальную колонку (после создания encoding)
    if unique_count > 10:  # Удаляем только если много уникальных значений
        df.drop(columns=[col], inplace=True, errors='ignore')
        print(f"✅ Обработан {col}: создан frequency encoding, удалена исходная колонка")
    else:
        print(f"✅ Обработан {col}: создан frequency encoding, исходная колонка сохранена")

print(f"\n✅ Обработано {len(categorical_cols)} категориальных признаков")
print(f"Создано {len(processed_categorical)} новых признаков из категориальных")


In [ ]:
print("\n--- ЗАДАЧА 30: Проверка выбросов ---\n")

# Выбираем только числовые колонки для анализа выбросов
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [col for col in numeric_cols if col not in ['target', 'user_id']]

print(f"Анализ выбросов для {len(numeric_cols)} числовых признаков\n")

# Методы обнаружения выбросов
outliers_info = []

for col in numeric_cols[:20]:  # Анализируем первые 20 признаков (чтобы не перегружать)
    if col not in df.columns:
        continue
    
    # Метод IQR (Interquartile Range)
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers_iqr = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()
    outliers_pct = (outliers_iqr / len(df)) * 100
    
    # Метод Z-score (выбросы > 3 стандартных отклонений)
    z_scores = np.abs((df[col] - df[col].mean()) / (df[col].std() + 1e-8))
    outliers_zscore = (z_scores > 3).sum()
    outliers_zscore_pct = (outliers_zscore / len(df)) * 100
    
    if outliers_iqr > 0 or outliers_zscore > 0:
        outliers_info.append({
            'column': col,
            'outliers_iqr': outliers_iqr,
            'outliers_iqr_pct': outliers_pct,
            'outliers_zscore': outliers_zscore,
            'outliers_zscore_pct': outliers_zscore_pct
        })

if len(outliers_info) > 0:
    outliers_df = pd.DataFrame(outliers_info)
    outliers_df = outliers_df.sort_values('outliers_iqr', ascending=False)
    
    print("ТОП-10 признаков с наибольшим количеством выбросов (IQR метод):")
    print("=" * 80)
    print(outliers_df.head(10).to_string(index=False))
    
    # Визуализация выбросов для топ-5 признаков
    top_5_cols = outliers_df.head(5)['column'].tolist()
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.flatten()
    
    for i, col in enumerate(top_5_cols[:6]):
        if col in df.columns:
            # Box plot для визуализации выбросов
            axes[i].boxplot(df[col].dropna(), vert=True, patch_artist=True,
                          boxprops=dict(facecolor='lightblue', alpha=0.7))
            axes[i].set_title(f'{col[:40]}...\nВыбросов: {outliers_df[outliers_df["column"]==col]["outliers_iqr"].values[0]:.0f}', 
                           fontsize=10, fontweight='bold')
            axes[i].set_ylabel('Значение')
            axes[i].grid(alpha=0.3, axis='y')
    
    # Скрываем лишние subplot'ы
    for i in range(len(top_5_cols), 6):
        axes[i].axis('off')
    
    plt.suptitle('Визуализация выбросов (топ-5 признаков)', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("\n⚠️ ВНИМАНИЕ: Выбросы могут быть важными для обнаружения мошенничества!")
    print("   Рекомендуется НЕ удалять выбросы, а использовать модели, устойчивые к ним")
    print("   (например, Random Forest, XGBoost, LightGBM)")
else:
    print("✅ Выбросов не обнаружено (или все признаки уже обработаны)")

print("\n✅ Проверка выбросов завершена")


In [ ]:
print("\n--- ЗАДАЧА 31: Проверка масштабирования признаков ---\n")

# Проверяем, какие признаки уже масштабированы (должны быть в диапазоне [0, 1] или около того)
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [col for col in numeric_cols if col not in ['target', 'user_id']]

# Исключаем бинарные и циклические признаки
cols_to_check = [col for col in numeric_cols 
                 if not any(x in col.lower() for x in ['_sin', '_cos', 'is_', 'changed', 'hour', 'day_of_week', 'day_of_month', 'month'])]

print(f"Проверка масштабирования для {len(cols_to_check)} признаков\n")

# Проверяем диапазоны значений
scaling_status = []
for col in cols_to_check[:15]:  # Проверяем первые 15
    if col in df.columns:
        min_val = df[col].min()
        max_val = df[col].max()
        mean_val = df[col].mean()
        
        # Если значения в диапазоне [0, 1] или близко к нему - вероятно масштабировано
        is_scaled = (min_val >= -0.1 and max_val <= 1.1) or (abs(mean_val) < 1 and abs(min_val) < 10 and abs(max_val) < 10)
        
        scaling_status.append({
            'column': col,
            'min': min_val,
            'max': max_val,
            'mean': mean_val,
            'likely_scaled': is_scaled
        })

if len(scaling_status) > 0:
    scaling_df = pd.DataFrame(scaling_status)
    print("Статус масштабирования признаков:")
    print("=" * 80)
    print(scaling_df.to_string(index=False))
    
    scaled_count = scaling_df['likely_scaled'].sum()
    print(f"\n✅ Вероятно масштабировано: {scaled_count} из {len(scaling_df)} проверенных признаков")
else:
    print("⚠️ Не удалось проверить масштабирование")

print("\n✅ Проверка масштабирования завершена")


## ЗАДАЧА 32: Сохранить features.csv

**Примечание:** Эта задача уже выполнена в `01_full_data_pipeline.ipynb`.
Здесь проверяем финальный файл.


In [ ]:
print("\n--- ЗАДАЧА 32: Проверка сохраненного features.csv ---\n")

FEATURES_OUTPUT = '../data/processed/transactions_with_features.csv'

if os.path.exists(FEATURES_OUTPUT):
    # Проверяем размер файла
    file_size = os.path.getsize(FEATURES_OUTPUT) / (1024 * 1024)  # в MB
    
    print(f"✅ Файл features.csv существует: {FEATURES_OUTPUT}")
    print(f"   Размер файла: {file_size:.2f} MB")
    print(f"   Размер данных: {df.shape}")
    print(f"   Количество признаков (без target): {len([c for c in df.columns if c != 'target'])}")
    
    # Проверяем наличие target
    if 'target' in df.columns:
        target_dist = df['target'].value_counts()
        print(f"\n   Распределение target:")
        for val, count in target_dist.items():
            print(f"     Класс {val}: {count} ({count/len(df)*100:.2f}%)")
    
    print(f"\n✅ Файл features.csv готов для передачи ML инжинеру!")
else:
    print(f"❌ Файл {FEATURES_OUTPUT} не найден!")
    print("   Сохраняем сейчас...")
    
    output_dir = os.path.dirname(FEATURES_OUTPUT)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)
    
    df.to_csv(FEATURES_OUTPUT, index=False)
    print(f"✅ Файл сохранен: {FEATURES_OUTPUT}")


## ЗАДАЧА 34: Написать preprocessing pipeline


In [ ]:
print("\n--- ЗАДАЧА 34: Создание preprocessing pipeline ---\n")

# Создаем функцию для preprocessing pipeline
# Это будет полезно для ML инжинера при обучении модели

def create_preprocessing_pipeline(df_train, df_test=None):
    """
    Создает preprocessing pipeline для обучения модели.
    
    Параметры:
    - df_train: тренировочный датасет
    - df_test: тестовый датасет (опционально)
    
    Возвращает:
    - X_train, y_train: признаки и целевая переменная для обучения
    - X_test, y_test: признаки и целевая переменная для теста (если передан)
    """
    
    # Копируем данные
    train = df_train.copy()
    
    # Разделяем на признаки и целевую переменную
    if 'target' in train.columns:
        y_train = train['target'].copy()
        X_train = train.drop(columns=['target'], errors='ignore')
    else:
        y_train = None
        X_train = train.copy()
    
    # Удаляем служебные колонки
    cols_to_drop = ['user_id', 'timestamp'] if 'user_id' in X_train.columns else []
    X_train = X_train.drop(columns=cols_to_drop, errors='ignore')
    
    # Заполняем пропуски
    X_train = X_train.fillna(0)
    
    # Обработка тестового датасета (если передан)
    if df_test is not None:
        test = df_test.copy()
        
        if 'target' in test.columns:
            y_test = test['target'].copy()
            X_test = test.drop(columns=['target'], errors='ignore')
        else:
            y_test = None
            X_test = test.copy()
        
        X_test = X_test.drop(columns=cols_to_drop, errors='ignore')
        X_test = X_test.fillna(0)
        
        # Убеждаемся, что колонки совпадают
        missing_cols = set(X_train.columns) - set(X_test.columns)
        if missing_cols:
            for col in missing_cols:
                X_test[col] = 0
        
        X_test = X_test[X_train.columns]  # Приводим к тому же порядку
        
        return X_train, y_train, X_test, y_test
    
    return X_train, y_train

# Пример использования pipeline
print("Создана функция create_preprocessing_pipeline()")
print("\nПример использования:")
print("=" * 60)
print("""
# Разделение на train/test
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['target'])

# Применение preprocessing pipeline
X_train, y_train, X_test, y_test = create_preprocessing_pipeline(train_df, test_df)

# Теперь можно обучать модель:
# from lightgbm import LGBMClassifier
# model = LGBMClassifier()
# model.fit(X_train, y_train)
# predictions = model.predict(X_test)
""")

# Тестируем pipeline на текущих данных
if 'target' in df.columns:
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['target'])
    
    X_train, y_train, X_test, y_test = create_preprocessing_pipeline(train_df, test_df)
    
    print(f"\n✅ Pipeline протестирован:")
    print(f"   X_train shape: {X_train.shape}")
    print(f"   X_test shape: {X_test.shape}")
    print(f"   y_train distribution: {y_train.value_counts().to_dict()}")
    print(f"   y_test distribution: {y_test.value_counts().to_dict()}")
    print(f"\n✅ Preprocessing pipeline готов к использованию!")
else:
    print("\n⚠️ Колонка 'target' не найдена, pipeline не протестирован")

print("\n✅ Задача 34 выполнена: preprocessing pipeline создан")


In [ ]:
print("\n--- ЗАДАЧА 36: Финальный набор признаков ---\n")

# Получаем список всех признаков (без target и user_id)
feature_cols = [col for col in df.columns if col not in ['target', 'user_id']]

print(f"📊 ФИНАЛЬНЫЙ НАБОР ПРИЗНАКОВ: {len(feature_cols)} признаков\n")
print("=" * 80)

# Группируем признаки по категориям
temporal_features = [col for col in feature_cols if any(x in col for x in ['hour', 'day', 'month', 'weekend', 'night', '_sin', '_cos'])]
user_features = [col for col in feature_cols if 'user_' in col or 'amount_diff' in col or 'amount_ratio' in col]
rolling_features = [col for col in feature_cols if any(x in col for x in ['tx_count', 'tx_mean', 'tx_std', 'tx_sum', 'tx_max'])]
statistical_features = [col for col in feature_cols if any(x in col for x in ['zscore', 'percentile', 'ratio', 'cv_amount'])]
device_features = [col for col in feature_cols if any(x in col for x in ['device', 'os', 'ip', 'geo'])]
behavioral_features = [col for col in feature_cols if any(x in col for x in ['time_since', 'time_until', 'rate', 'interval', 'rapid', 'anomaly', 'morning', 'afternoon', 'evening'])]
categorical_features = [col for col in feature_cols if '_freq' in col or '_target_enc' in col]
other_features = [col for col in feature_cols if col not in temporal_features + user_features + rolling_features + statistical_features + device_features + behavioral_features + categorical_features]

categories = {
    'Временные признаки': temporal_features,
    'Признаки по пользователю': user_features,
    'Rolling-window признаки': rolling_features,
    'Статистические признаки': statistical_features,
    'Device/OS/IP признаки': device_features,
    'Признаки поведения': behavioral_features,
    'Категориальные encoding': categorical_features,
    'Другие признаки': other_features
}

# Выводим статистику по категориям
print("Распределение признаков по категориям:\n")
for category, cols in categories.items():
    if len(cols) > 0:
        print(f"  {category}: {len(cols)} признаков")

print(f"\n  ВСЕГО: {len(feature_cols)} признаков")

# Сохраняем список признаков в файл
features_list_file = '../data/processed/features_list.txt'
with open(features_list_file, 'w', encoding='utf-8') as f:
    f.write("ФИНАЛЬНЫЙ НАБОР ПРИЗНАКОВ ДЛЯ ML МОДЕЛИ\n")
    f.write("=" * 80 + "\n\n")
    f.write(f"Всего признаков: {len(feature_cols)}\n\n")
    
    for category, cols in categories.items():
        if len(cols) > 0:
            f.write(f"{category} ({len(cols)}):\n")
            for col in cols:
                f.write(f"  - {col}\n")
            f.write("\n")

print(f"\n✅ Список признаков сохранен в: {features_list_file}")

# Выводим первые 20 признаков для примера
print("\nПримеры признаков (первые 20):")
for i, col in enumerate(feature_cols[:20], 1):
    print(f"  {i:2d}. {col}")
if len(feature_cols) > 20:
    print(f"  ... и еще {len(feature_cols) - 20} признаков")

print(f"\n✅ Финальный набор из {len(feature_cols)} признаков готов для ML модели!")


In [ ]:
print("\n" + "=" * 80)
print("ИТОГОВЫЙ ОТЧЕТ ПО FEATURE ENGINEERING")
print("=" * 80)

print("\n✅ ВЫПОЛНЕННЫЕ ЗАДАЧИ:")
print("  29. Обработка категориальных признаков - ВЫПОЛНЕНО")
print("  30. Проверка выбросов - ВЫПОЛНЕНО")
print("  31. Нормирование/масштабирование - ВЫПОЛНЕНО (в 01_full_data_pipeline.ipynb)")
print("  32. Сохранение features.csv - ВЫПОЛНЕНО (в 01_full_data_pipeline.ipynb)")
print("  33. Создание notebook 02_feature_engineering.ipynb - ВЫПОЛНЕНО")
print("  34. Preprocessing pipeline - ВЫПОЛНЕНО")
print("  36. Финальный набор признаков - ВЫПОЛНЕНО")

print("\n📊 ФИНАЛЬНАЯ СТАТИСТИКА:")
print(f"  - Всего признаков: {len([c for c in df.columns if c != 'target'])}")
print(f"  - Размер датасета: {df.shape}")
if 'target' in df.columns:
    target_dist = df['target'].value_counts()
    print(f"  - Распределение классов: {target_dist.to_dict()}")

print("\n📁 ФАЙЛЫ ДЛЯ ML ИНЖИНЕРА:")
print("  - data/processed/transactions_with_features.csv - финальный датасет")
print("  - data/processed/features_list.txt - список всех признаков")
print("  - FEATURES_DOCUMENTATION.md - документация по признакам")
print("  - notebooks/02_feature_engineering.ipynb - preprocessing pipeline")

print("\n✅ ВСЕ ЗАДАЧИ FEATURE ENGINEERING ВЫПОЛНЕНЫ!")
print("   Датасет готов для передачи ML инжинеру. 🚀")
